In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import tensorflow as tf
import plotly.graph_objs as go
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [16]:
df = yf.download('JPY=X','2001-01-01','2020-08-10',interval='1d')

[*********************100%***********************]  1 of 1 completed


In [17]:
df.to_csv("C:/Users/Dell/Desktop/goldo3.csv")

In [18]:
df = pd.read_csv("C:/Users/Dell/Desktop/goldo3.csv")
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2001-01-01,114.610001,114.629997,114.269997,114.570000,114.570000,0
1,2001-01-02,114.610001,115.050003,114.209999,114.400002,114.400002,0
2,2001-01-03,114.400002,114.669998,113.500000,113.839996,113.839996,0
3,2001-01-04,113.809998,116.110001,113.580002,116.089996,116.089996,0
4,2001-01-05,116.070000,116.870003,115.940002,116.449997,116.449997,0


In [19]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [20]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.7
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

3560
1527


In [21]:
from keras.preprocessing.sequence import TimeseriesGenerator
look_back = 10

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=32)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=32)

In [22]:
model = Sequential()
model.add(
    LSTM(50,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
history=model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25
111/111 [==============================] - 1s 11ms/step - loss: 2188.3291
Epoch 2/25
111/111 [==============================] - 1s 7ms/step - loss: 1.7974
Epoch 3/25
111/111 [==============================] - 1s 8ms/step - loss: 1.5248
Epoch 4/25
111/111 [==============================] - 1s 7ms/step - loss: 1.3071
Epoch 5/25
111/111 [==============================] - 1s 8ms/step - loss: 0.8380
Epoch 6/25
111/111 [==============================] - 1s 8ms/step - loss: 0.7989
Epoch 7/25
111/111 [==============================] - 1s 8ms/step - loss: 0.7399
Epoch 8/25
111/111 [==============================] - 1s 9ms/step - loss: 0.7049
Epoch 9/25
111/111 [==============================] - 1s 8ms/step - loss: 0.6793
Epoch 10/25
111/111 [==============================] - 1s 8ms/step - loss: 0.6990
Epoch 11/25
111/111 [==============================] - 1s 8ms/step - loss: 0.6889
Epoch 12/25
111/111 [==============================] - 1s 8ms/step - loss: 0.7105
Epoch 13/25
111/111 [

In [30]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "USD/JPY",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [31]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [32]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)    
trace3 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'forecast'

)

layout = go.Layout(
    
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()